# NB-04 Evaluation 

### In this notebook I take the responses that we gathered from the LLM and evaluate how successfully my project achieved the spec task

In [1]:
import pickle
import os 
import re 
import pandas as pd
import numpy as np

In [7]:
llm_responses = pickle.load(open("llm_responses.pkl", "rb"))
llm_responses_empty_countries = pickle.load(open("llm_responses_non_retrieved_countries.pkl", "rb"))
complete_embeddings_df = pickle.load(open("complete_embeddings_df.pkl", "rb"))

In [8]:
import re 
import pandas as pd

all_llm_responses = {**llm_responses, **llm_responses_empty_countries}

llm_df = pd.DataFrame([
    {"country": country, "llm_response": response}
    for country, response in all_llm_responses.items()
])

llm_df = pd.DataFrame([{"country": country, "llm_response": response} for country, response in all_llm_responses.items()])

def extract_citations(text):
    pattern = r"\[Doc ID: (.*?), Chunk ID: (\d+)\]"
    return re.findall(pattern, text)

llm_df["citations"] = llm_df["llm_response"].apply(extract_citations)

# We drop any rows if they do not have citations 
exploded = llm_df.explode("citations").dropna(subset=["citations"])

exploded[["doc_id", "chunk_index"]] = pd.DataFrame(exploded["citations"].tolist(), index=exploded.index)
exploded["chunk_index"] = exploded["chunk_index"].astype(int)

merged_df = pd.merge(exploded, complete_embeddings_df,on=["doc_id", "chunk_index"],how="left")

merged_df = merged_df.drop(columns=["citations"])

In [10]:
# We eneded up with 59 countries for which our robust RAG pipeline delivered results
len(merged_df['country_x'].unique())

59

In [13]:
merged_df['chunk_metadata'].iloc[0]

{'links': [],
 'filename': 'bangladesh_english_20220601.pdf',
 'filetype': 'application/pdf',
 'languages': ['eng'],
 'parent_id': '663ea1bfffe5038f3f0cf667f14c4257',
 'coordinates': {'points': [[265.25, 541.20112],
   [265.25, 605.16112],
   [514.08088, 605.16112],
   [514.08088, 541.20112]],
  'system': '<unstructured.documents.coordinates.PixelSpace object at 0x2b8bbd790>'},
 'page_number': 30,
 'last_modified': '2025-03-12T10:27:13',
 'file_directory': '../data/pdfs',
 '_known_field_names': ['page_number',
  'parent_id',
  'links',
  'filetype',
  'last_modified',
  'link_texts',
  'attached_to_filename',
  'sent_to',
  'url',
  'link_urls',
  'regex_metadata',
  'coordinates',
  'section',
  'sent_from',
  'data_source',
  'text_as_html',
  'emphasized_text_tags',
  'category_depth',
  'header_footer_type',
  'detection_class_prob',
  'filename',
  'languages',
  'subject',
  'emphasized_text_contents',
  'detection_origin',
  'image_path',
  'is_continuation',
  'page_name',
  'f

In [ ]:
def extract_metadata_fields(metadata):
    try:
        page = metadata.get("page_number")
        coords = metadata.get("coordinates", {}).get("points")
        filename = metadata.get("filename")
        return pd.Series([page, coords, filename])
    except Exception:
        return pd.Series([None, None, None])

merged_df[["page_number", "coordinate_points", "source_filename"]] = merged_df["chunk_metadata"].apply(extract_metadata_fields)